In [1]:
from langchain_community.document_loaders import PyPDFLoader 
import os
from dotenv import load_dotenv
import pypdf
import PyPDF2
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
from nltk.corpus import wordnet
import pdfplumber
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import re
from transformers import pipeline


In [2]:
PyPDFLoader("/Users/mukulsaini/Downloads/merged_output.pdf")





In [3]:
def merge_pdfs(paths, output):
    merger = PyPDF2.PdfMerger()
    
    for path in paths:
        merger.append(path)
        
    merger.write(output)
    merger.close()

pdf_files = [
    "/Users/mukulsaini/Downloads/The-$30000-Bequest.pdf",
    "/Users/mukulsaini/Downloads/Dracula.pdf",
    "/Users/mukulsaini/Downloads/Crime-and-Punishment-.pdf"
]

output_pdf = "/Users/mukulsaini/Downloads/merged_output.pdf"

merge_pdfs(pdf_files, output_pdf)


In [4]:

Loader = PyPDFLoader("/Users/mukulsaini/Downloads/merged_output.pdf")
docs = Loader.load()
#docs

In [5]:



def generate_unique_id():
    global id_counter
    id_counter += 1
    return id_counter


id_counter = 0


pdf_path = "/Users/mukulsaini/Downloads/merged_output.pdf"


pdf_reader = PdfReader(pdf_path)


nodes = []


for page_num, page in enumerate(pdf_reader.pages):
    text = page.extract_text()
    if text is None:
        continue

    lines = text.split("\n")

    
    current_chapter = None
    current_section = None
    current_subsection = None

    for line in lines:
        line = line.strip()

        
        if line.startswith("Chapter"):
            current_chapter = line
            node_id = generate_unique_id()
            nodes.append({'NodeID': node_id, 'ParentID': None, 'NodeType': 'chapter', 'NodeName': line, 'Text': ''})
            current_chapter_id = node_id
            current_section = None  # Reset sections and subsections
            current_subsection = None

        
        elif line.startswith("Section") and current_chapter:
            current_section = line
            node_id = generate_unique_id()
            nodes.append({'NodeID': node_id, 'ParentID': current_chapter_id, 'NodeType': 'section', 'NodeName': line, 'Text': ''})
            current_section_id = node_id
            current_subsection = None

        
        elif line.startswith("Subsection") and current_section:
            current_subsection = line
            node_id = generate_unique_id()
            nodes.append({'NodeID': node_id, 'ParentID': current_section_id, 'NodeType': 'subsection', 'NodeName': line, 'Text': ''})
            current_subsection_id = node_id

        
        elif line and current_subsection:
            node_id = generate_unique_id()
            nodes.append({'NodeID': node_id, 'ParentID': current_subsection_id, 'NodeType': 'paragraph', 'NodeName': '', 'Text': line})
        elif line and current_section:
            node_id = generate_unique_id()
            nodes.append({'NodeID': node_id, 'ParentID': current_section_id, 'NodeType': 'paragraph', 'NodeName': '', 'Text': line})
        elif line and current_chapter:
            node_id = generate_unique_id()
            nodes.append({'NodeID': node_id, 'ParentID': current_chapter_id, 'NodeType': 'paragraph', 'NodeName': '', 'Text': line})


df = pd.DataFrame(nodes)


df['Level'] = df['NodeType'].map({
    'chapter': 1,
    'section': 2,
    'subsection': 3,
    'paragraph': 4
})


#print(df)


df.to_csv('/Users/mukulsaini/Downloads/hierarchical_index.csv', index=False)


In [6]:
pdf_path = "/Users/mukulsaini/Downloads/merged_output.pdf"
text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        extracted_text = page.extract_text()
        if extracted_text:  # Check if the page has text
            text += extracted_text


tokens = word_tokenize(text)


def get_synonyms(word):
    synonyms = set()
  
    for syn in wordnet.synsets(word):
       
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
   
    synonyms.discard(word)
    return synonyms


expanded_tokens = set()  
for token in tokens:
    if token.isalpha():  
        synonyms = get_synonyms(token.lower())  
        if synonyms:
            expanded_tokens.update(synonyms)
        else:
            expanded_tokens.add(token.lower())


cleaned_tokens = {re.sub(r'\W+', '', token) for token in expanded_tokens if token.isalpha()}


with open("/Users/mukulsaini/Downloads/expanded_output.txt", "w") as file:
    file.write(" ".join(cleaned_tokens))

print("Synonym extraction and saving complete.")


Synonym extraction and saving complete.


In [7]:
with open("/Users/mukulsaini/Downloads/expanded_output.txt", "r") as file:
    content = file.read()

print("File Contents:")
#print(content)

File Contents:


In [8]:
pdf_path = "/Users/mukulsaini/Downloads/merged_output.pdf"
with pdfplumber.open(pdf_path) as pdf:
    text = "".join(page.extract_text() for page in pdf.pages)


retriever = pipeline("question-answering")


question = "Question"
result = retriever(question=question, context=text)
print(result)


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'score': 0.8612107634544373, 'start': 1863401, 'end': 1863423, 'answer': 'Pulcheria Alexandrovna'}


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(metadata={'source': '/Users/mukulsaini/Downloads/merged_output.pdf', 'page': 1}, page_content="Project\tGutenberg's\tThe\t$30,000\tBequest\tand\tOther\tStories,\tby\tMark\tTwain\nThis\teBook\tis\tfor\tthe\tuse\tof\tanyone\tanywhere\tat\tno\tcost\tand\twith\nalmost\tno\trestrictions\twhatsoever.\t\tYou\tmay\tcopy\tit,\tgive\tit\taway\tor\nre-use\tit\tunder\tthe\tterms\tof\tthe\tProject\tGutenberg\tLicense\tincluded\nwith\tthis\teBook\tor\tonline\tat\twww.gutenberg.org\nTitle:\tThe\t$30,000\tBequest\tand\tOther\tStories\nAuthor:\tMark\tTwain\nRelease\tDate:\tMay\t12,\t2009\t[EBook\t#142]\nLast\tUpdated:\tFebruary\t24,\t2018\nLanguage:\tEnglish\n***\tSTART\tOF\tTHIS\tPROJECT\tGUTENBERG\tEBOOK\t$30,000\tBEQUEST\tAND\tOTHERS\t***\nProduced\tby\tAn\tAnonymous\tVolunteer,\tand\tDavid\tWidger"),
 Document(metadata={'source': '/Users/mukulsaini/Downloads/merged_output.pdf', 'page': 2}, page_content='THE\t$30,000\tBEQUEST'),
 Document(metadata={'source': '/Users/mukulsaini/Downloads/me

In [10]:
documents=text_splitter.split_documents(docs)
#documents

In [11]:

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:30],OllamaEmbeddings())

In [12]:
db


In [13]:
query="Mukul "
result=db.similarity_search(query)
result[0].page_content

'THE\t$30,000\tBEQUEST'

In [14]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")
llm

Ollama()

In [15]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [17]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x496954290>)

In [18]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [19]:
response=retrieval_chain.invoke({"input":"What is coal"})

In [20]:
response['answer']

'Based on the provided context, it seems that "coal" is being referred to as a potential investment opportunity. The speaker is discussing with Aleck how they will organize and invest their $30,000 bequest, with the goal of earning a noble income. The speaker mentions that they will put in ten shares for one, and when they organize, they\'ll get three shares for one. They also mention that the shares will be worth about $30,000 in a year, and they will pay ten percent half-yearly.\n\nTherefore, the answer to the question "What is coal?" based on the context is: Coal is a potential investment opportunity mentioned in the conversation between the speaker and Aleck.'